We will need to install opencv for python first and foremost

In [62]:
import cv2
import os
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle



Next up we can define our classifier we will be using, including our nicolas cage dataset

In [55]:
#Model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Load our Nicolas dataset
cage_directory = "data/cage/Nicolas Cage Face - Frontal"
person_directory = "data/person/person face picture - Frontal view"
image_size = (100, 100)
n_neighbours = 9


Pre-Process Image

In [21]:
def preprocess_image(image):
    greyscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(greyscale, image_size)
    
    return resized



Extract features

In [40]:
def extract_features(image):
    win_size = (128, 128) 
    block_size = (16, 16)
    block_stride = (8,8)
    cell_size = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    hog = cv2.HOGDescriptor(win_size,block_size, block_stride, cell_size, nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    
    
    winStride = (8,8)
    padding = (8,8)
    locations = ((10,20),)
    hist = hog.compute(image, winStride=winStride, padding=padding, locations=locations)
    print(hist)
    return hist 



Training Data

In [66]:

X_train = []
y_train = []
#Its late, im lazy, sorry Sarina,
# So it seems the knn trains faces, whichever set is first is most likely to be predicted
#Quik shuffle
for filename in os.listdir(person_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        #print(f"{filename}")
        image_path = os.path.join(person_directory, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image)
        features = extract_features(preprocessed_image)
        X_train.append(features)
        y_train.append("Normal Person")
for filename in os.listdir(cage_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        #print(f"{filename}")
        image_path = os.path.join(cage_directory, filename)
        image = cv2.imread(image_path)
        preprocessed_image = preprocess_image(image)
        features = extract_features(preprocessed_image)
        X_train.append(features)
        y_train.append("Nicolas Cage")
        


X_train, y_train = shuffle(X_train, y_train, random_state=32)  
X_train = np.array(X_train)
#Train KNN Model
model = KNeighborsClassifier(n_neighbors=n_neighbours)
model.fit(X_train, y_train)
model.score(X_train, y_train)


[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0.

0.49748743718592964

In [59]:
test_image_dir = 'data/cage/Nicolas Cage normal front face picture'
#test_image_dir = 'data/person/person face picture - Frontal view'

In [67]:
#Loop through test images

for filename in os.listdir(test_image_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        test_image_path = os.path.join(test_image_dir, filename)
        test_image = cv2.imread(test_image_path)
        
        #preprocess
        preprocessed = preprocess_image(test_image)
        
        #Face detection
        faces = face_cascade.detectMultiScale(preprocessed, 1.1, 4)
        
        #predict logic for each image
        prediction =""
        if len(faces) == 0:
            prediction = "No Face Detected"
        else:
            #extract features for the detected face
            face_region = preprocessed[faces[0][1]:faces[0][1]+faces[0][3], faces[0][0]:faces[0][0]+faces[0][2]]
            test_features = extract_features(face_region)
            prediction = model.predict([test_features])[0]
            
        print(f"Image: {filename} - Prediction: {prediction}")

[0. 0. 0. ... 0. 0. 0.]
Image: Image_1.jpg - Prediction: Nicolas Cage
[0. 0. 0. ... 0. 0. 0.]
Image: Image_10.jpg - Prediction: Nicolas Cage
[0. 0. 0. ... 0. 0. 0.]
Image: Image_2.jpg - Prediction: Nicolas Cage
[0. 0. 0. ... 0. 0. 0.]
Image: Image_3.jpg - Prediction: Nicolas Cage
Image: Image_4.jpg - Prediction: No Face Detected
[0. 0. 0. ... 0. 0. 0.]
Image: Image_5.jpg - Prediction: Nicolas Cage
[0. 0. 0. ... 0. 0. 0.]
Image: Image_6.jpg - Prediction: Nicolas Cage
Image: Image_7.jpg - Prediction: No Face Detected
Image: Image_8.jpeg - Prediction: No Face Detected
[0. 0. 0. ... 0. 0. 0.]
Image: Image_9.jpg - Prediction: Nicolas Cage
